**Inviscid Burgers’ Equation**

For a given field $u(x, t)$ and diffusion coefficient (or kinematic viscosity, as in the original fluid mechanical context)  $\nu$ , the general form of Burgers' equation (also known as viscous Burgers' equation) in one space dimension is the dissipative system:

Burger's equation is the following:

$$\frac{\partial u(x, t)}{\partial t}+u \frac{\partial u(x, t)}{\partial x}=\nu \frac{\partial^{2} u(x, t)}{\partial x^{2}}$$
A special solution to the above PDE is:

$$u(x, t)=1-\tanh \frac{x-x_{c}-t}{2 \nu}$$

here i define $x_c$ = 0:
$$u(x, t)=1-\tanh \frac{x-t}{2 \nu}$$

\begin{array}{c}
\frac{\partial u}{\partial t}+u \frac{\partial u}{\partial x}=\nu \frac{\partial^{2} u(x, t)}{\partial x^{2}}, (x,t)∈[0,1] × [0,1]  \\
u(x, 0)=F(x)=1-\tanh \frac{x}{2 \nu}, x∈[0,1]\\
u(x,t) = g(x,t) = 1-\tanh \frac{x-t}{2 \nu}, x ∈ \{0, 1\}
\end{array}


## 1.Setup

In [1]:
import torch
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import matplotlib.pyplot as plt
from math import *
import time
import copy


# %%
torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
# check pytorch version
print(torch.__version__)

1.13.1+cu117



## 2.Hyperparameter

In [4]:
dimension = 2
input_width,layer_width = dimension, 300
layer_depth = 5
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")


## 3.Neural Architecture

In [5]:
# %%
# activation function
def activation(x):
    return torch.mul(x, torch.sigmoid(x))


class Net(torch.nn.Module):
    def __init__(self,input_width,layer_width,layer_depth):
        super(Net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        for i in range(layer_depth-1):
            setattr(self, 'layer'+str(i), torch.nn.Linear(layer_width, layer_width))
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x,layer_depth):
        y = activation(self.layer_in(x))
        for i in range(layer_depth-1):
            y = activation(getattr(self, 'layer'+str(i))(y))
        output = self.layer_out(y)
        return output


${\rm N}(\vec{x}) := {\rm A}_{\rm 4}(\sigma{\rm A}_3({\sigma}{\rm A}_2({\sigma}{\rm A}_{\rm 1}(\vec{x}))))\ ,$, where


${\rm A}_{\rm 1}: \mathbb{R}^2{→}\mathbb{R}^w$, ${\rm A}_2: \mathbb{R}^w{→}\mathbb{R}^w$, ${\rm A}_3: \mathbb{R}^w{→}\mathbb{R}^w$, ${\rm A}_{\rm 4}: \mathbb{R}^w{→}\mathbb{R}$, 

note: there are $2w^2+6w+1$ parameters need to be trained in total in ${\rm N}(x)$

## 4.Training setup

### 4.1 Initialize the weights using Xavier normal initialization

In [6]:
# %%
# Xavier normal initialization for weights:
#             mean = 0 std = gain * sqrt(2 / fan_in + fan_out)
# zero initialization for biases
def initialize_weights(self):
    for m in self.modules():
        if isinstance(m,nn.Linear):
            nn.init.xavier_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()

### 4.2 Define $u_0(x)$

$$ [0,1] {\ni} x {↦} u_0(x) =1- \tanh \frac{x}{2 \nu}{∈} {\mathbb R} $$

In [7]:
# Define u_0
def u_0x(X):
    x = X[:,0]
    temp1 = torch.sub(1,torch.tanh(x/(2*nu)))
    u_temp = temp1
    return u_temp.reshape([X.size()[0], 1]).to(device)

### 4.3 Define the exact solution

$$[0,1]{×}[0,1]{\ni}(x, t) {↦} u(x, t)=1-\tanh \frac{x-t}{2 \nu} {∈} {\mathbb R}$$


In [8]:
# defination of exact solution
def u_ex(X):     
    x = X[:,0]
    t = X[:,1]
    
    temp1 = torch.sub(1,torch.tanh((x-t)/(2*nu)))
    u_temp = temp1
    return u_temp.reshape([x.size()[0], 1]).to(device)

### 4.4 Define g(x,t)

$$ \{0,1\}×[0,1] \ni (x,t) ↦ g(x,t) = 1-\tanh \frac{x-t}{2 \nu} ∈ {\mathbb R}$$

In [9]:
def g_x(X):     
    x = X[:,0].to(device)
    t = X[:,1].to(device)
    temp1 = torch.sub(1,torch.tanh((x-t)/(2*nu)))
    u_temp = temp1
    return u_temp.reshape([x.size()[0], 1]).to(device)

In [10]:
def g_0x(X):
    x = torch.zeros([X.shape[0],1])[:,0].to(device)
    x.requires_grad = True
    t = X[:,1].to(device)
    temp1 = torch.sub(1,torch.tanh((x-t)/(2*nu)))
    u_temp = temp1
    return u_temp.reshape([x.size()[0], 1]).to(device)

In [11]:
def g_1x(X):
    x = torch.ones([X.shape[0],1])[:,0].to(device)
    x.requires_grad = True
    t = X[:,1].to(device)
    temp1 = torch.sub(1,torch.tanh((x-t)/(2*nu)))
    u_temp = temp1
    return u_temp.reshape([x.size()[0], 1]).to(device)

### 4.4 Build the model

$x = {\vec X}[:,0]$

$t = {\vec X}[:,1]$

Model for training on initial and boundary:$${\rm model}({x,t}) := {\rm N}(x, t)  $$
Model for training on boundary:$${\rm model}({x,t}) := {\rm N}(x, t) ⋅ x ⋅ (1-x)  +  (1-x) ⋅ g(0,t) + x{⋅} g(1,t)$$
Model for training on initial:$${\rm model}({x,t}) := {\rm N}(x, t)  ⋅ t  +  u_0(x) \cdot (1-t)$$



In [12]:
def model(X,initial = False, boundary = False):
    x = X[:,0]
    t = X[:,1].reshape([x.size()[0], 1])
    x_temp = x.reshape([x.size()[0], 1])
    t_temp = t.reshape([t.size()[0], 1])
    x_minus = (1.0-x).reshape([x.size()[0], 1])
    t_minus = (1.0-t).reshape([t.size()[0], 1])

    if initial == False and boundary == False:
        term1 = net(X,layer_depth)
        model_u_temp = term1
    elif initial == False and boundary == True:
        # term1 = net(X,layer_depth) * x_temp * x_minus
        term1 = torch.mul(net(X,layer_depth),torch.mul(x_temp,x_minus))
        term2 = torch.mul(g_0x(X),x_minus)
        term3 = torch.mul(g_1x(X),x_temp)
        model_u_temp =  torch.add(torch.add(term1,term2),term3)
    else:
        term1 = torch.mul(net(X,layer_depth),t)
        term2 = torch.mul(u_0x(X),t_minus)
        model_u_temp =  torch.add(term1,term2)
                   
    return model_u_temp.reshape([x.size()[0], 1]).to(device)

#### 4.5.1 $ν_1$ generator

Generate random points $(x_n, t_n)$ from $[0,1] × [0,1]$

In [13]:
# generate points by random
def generate_sample_one(data_size):
    sample_temp = torch.rand(data_size, dimension)
    sample_temp.requires_grad = True
    return sample_temp.to(device)

#### 4.5.2 $ν_2$ generator

Generate random points $x_n$ from $[0,1]$

In [14]:
def generate_sample_two(data_size):
    sample_temp = torch.rand(data_size, dimension)
    sample_temp[:,1] = 0
    sample_temp.requires_grad = True
    return sample_temp.to(device)

#### 4.5.3 $ν_3$ generator

Generate random points $x_n$ from ${\{0,1\}}$

In [15]:
def generate_sample_three(data_size):
    sample_temp = torch.rand(data_size, dimension)
    sample_temp[0: len(sample_temp)//2, 0] = 0
    sample_temp[len(sample_temp)//2: len(sample_temp), 0] = 1
    sample_temp.requires_grad = True
    return sample_temp.to(device)

### 4.6 Define Loss function

Loss for training on initial and boundary:$$Loss(û) = ||{\frac{∂û}{∂t}} +  \hat{u} {\frac{∂û}{∂x}} - \nu \frac{\partial^{2} û}{\partial x^{2}}||^2_{[0,1]{×}{[0,1]},{ν_1}} +||û-F(x)||^2_{t=0,[0,1], ν_2} + ||û-g(x,t)||^2_{\{0,1\}{×}{[0,1]}, ν_3}$$
Loss for training on boundary:$$Loss(û) = ||{\frac{∂û}{∂t}} +  \hat{u} {\frac{∂û}{∂x}} - \nu \frac{\partial^{2} û}{\partial x^{2}}||^2_{[0,1]{×}{[0,1]},{ν_1}} +||û-F(x)||^2_{t=0,[0,1], ν_2} $$

Loss for training on initial:$$Loss(û) = ||{\frac{∂û}{∂t}} +  \hat{u} {\frac{∂û}{∂x}} - \nu \frac{\partial^{2} û}{\partial x^{2}}||^2_{[0,1]{×}{[0,1]},{ν_1}} + ||û-g(x,t)||^2_{\{0,1\}{×}{[0,1]}, ν_3}$$

In [16]:

# loss function to DGM by auto differential
def loss_function(x1, x2, x3, initial=False, boundary=False):
    # first term
    u_hat = model(x1,initial, boundary)
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x1, grad_outputs = torch.ones(u_hat.shape).to(device), create_graph = True)  # dx and dt
    dx = grad_u_hat[0][:, 0]  # dx
    dt = grad_u_hat[0][:, 1]  # dt
    # dxx and dtt
    grad_dx = torch.autograd.grad(outputs = dx, inputs = x1, grad_outputs = torch.ones(dx.shape).to(device).to(device), create_graph = True)
    dxx = grad_dx[0][:, 0]
    dtt = grad_dx[0][:, 1]
    # loss1 = torch.sum((dt + torch.multiply(u_hat.T,dx))**2) / len(x1)
    loss1 = torch.add(dt, torch.mul(u_hat.T,dx))
    loss1 = torch.sub(loss1, torch.mul(nu, dxx))
    loss1 = torch.sum(torch.mul(loss1,loss1)) / len(x1)

    # bias = torch.square(torch.mean(dt + torch.multiply(u_hat.T,dx)))
    # variance = torch.mean(torch.square(dt + torch.multiply(u_hat.T,dx) - torch.mean(dt + torch.multiply(u_hat.T,dx))))
    #second term
    if initial == False:
        u_hat2 = model(x2,initial, boundary)
        u0 = u_0x(x2)
        loss2 = torch.sub(u_hat2, u0)
        loss2 = torch.sum(torch.mul(loss2,loss2)) / len(x2)
    else:
        loss2 = 0

    #third term

    if boundary == False:
        u_hat3 = model(x3,initial, boundary)
        u_ex3 = g_x(x3)
        loss3 = torch.sub(u_hat3, u_ex3)
        loss3 = torch.sum(torch.mul(loss3,loss3)) / len(x3)
    else:
        loss3 = 0
    return loss1 + loss2 + loss3
    # return loss

## 5.Doing the training

In [17]:
def plot_loss_epoch(loss_record_all, error_record_all,epoch, var_loss, var_error,nu):
    para = nu
    fig = plt.figure()
    # Set subplots size
    fig.set_figheight(15)
    fig.set_figwidth(20)

    ax = fig.add_subplot(2, 1, 1)
    # plot a smooth curve of loss
    ax.plot(np.arange(epoch), loss_record_all[0])
    ax.plot(np.arange(epoch), loss_record_all[1])
    ax.plot(np.arange(epoch), loss_record_all[2])
    # append error bar to the plot 
    ax.errorbar(np.arange(epoch), loss_record_all[0], yerr=var_loss[0], label='vanilla')
    ax.errorbar(np.arange(epoch), loss_record_all[1], yerr=var_loss[1], label='boundary-included')
    ax.errorbar(np.arange(epoch), loss_record_all[2], yerr=var_loss[2], label='initial-included')

    ax.set_xlabel('epoch')
    ax.set_ylabel('Empirical loss')
    ax.set_title("Empirical loss for 1D Burgers' equation, epoch = 30000, nu: {}, lr = 0.0001, data_size = 3000".format(para), fontsize=12)
    ax.set_yscale("log")
    ax.legend()

    ax = fig.add_subplot(2, 1, 2)
    # plot a smooth curve of loss
    ax.plot(np.arange(epoch), error_record_all[0])
    ax.plot(np.arange(epoch), error_record_all[1])
    ax.plot(np.arange(epoch), error_record_all[2])
    # append error bar to the plot
    ax.errorbar(np.arange(epoch), error_record_all[0], yerr=var_error[0], label='vanilla')
    ax.errorbar(np.arange(epoch), error_record_all[1], yerr=var_error[1], label='boundary-included')
    ax.errorbar(np.arange(epoch), error_record_all[2], yerr=var_error[2], label='initial-included')

    ax.set_xlabel('epoch')
    ax.set_ylabel('Fractional error w.r.t true solution')
    ax.set_title("Fractional error w.r.t true solution, epoch = 30000, nu: {}, lr = 0.0001,data_size = 3000".format(para), fontsize=12)
    ax.set_yscale("log")
    ax.legend()

    plt.savefig('loss_epoch,1D, epoch = 30000, nu: {}, width=300, lr = 0.0001,data_size = 3000.png'.format(para))



In [18]:

# Testing data
test_data = torch.zeros((101, 101, 2))
for i in range(101):
  x_test = i / 100
  for j in range(101):
    t_test = j / 100
    test_data[i][j][0] = x_test
    test_data[i][j][1] = t_test
test_data = torch.reshape(test_data, (101*101, 2)).to(device)



def cal_error(x,initial = True, boundary = True):
    u_hat = model(x,initial, boundary)
    u = u_ex(x)
    error = ((u_hat - u).norm(2))**2 / (u.norm(2))**2
    return error

In [ ]:
epoch = 30000
import pickle
# layer_width_list = [4,40,200,300,1000,2000,3000]
nu_list = [0.01,0.1]
print(layer_width)

for i in range(len(nu_list)):
  nu = nu_list[i]
  print("current nu_list: {}".format(nu))
  loss_record_all = [[],[],[]]
  error_record_all = [[],[],[]]
  for i in range(3):
    print("current run: {}".format(i))
    net = Net(input_width,layer_width, layer_depth).to(device)
    initialize_weights(net)
    optimizer = optim.Adam(net.parameters())
    # %%
    loss_record = np.zeros(epoch)
    error_record = np.zeros(epoch)
    data_size = 3000
    x1 = generate_sample_one(data_size//3)
    x2 = generate_sample_two(data_size//3)
    x3 = generate_sample_three(data_size//3)
    time_start = time.time()
    for i in range(epoch):
        # scale_y[i] = float(model(scale_x))
        optimizer.zero_grad()
        loss = loss_function(x1, x2, x3, initial = False, boundary=False)
        loss_record[i] = float(loss)
        loss.backward()
        optimizer.step() 
        error_record[i] = cal_error(test_data, initial=False, boundary=False)
        torch.cuda.empty_cache() # clear memory
        print("Now epoch number is:{}".format(i),end='\r')
        if i % 2500 == 0:
          # y2.append(model(x2))
          print("current epoch is: ", i)
          print("current loss is: ", loss.detach())
    time_end = time.time()
    print('total time is: ', time_end-time_start, 'seconds')
    # plot_heatmap("venilla",initial=False, boundary=False)
    loss_record_all[0].append(loss_record)
    error_record_all[0].append(error_record)

    net = Net(input_width,layer_width, layer_depth).to(device)
    initialize_weights(net)
    optimizer = optim.Adam(net.parameters())
    # %%
    loss_record = np.zeros(epoch)
    error_record = np.zeros(epoch)
    time_start = time.time()
    for i in range(epoch):
        # scale_y[i] = float(model(scale_x))
        optimizer.zero_grad()
        loss = loss_function(x1, x2, x3, initial = False, boundary=True)
        loss_record[i] = float(loss)
        loss.backward()
        optimizer.step() 
        error_record[i] = cal_error(test_data, initial=False, boundary=True)
        torch.cuda.empty_cache() # clear memory
        print("Now epoch number is:{}".format(i),end='\r')
        if i % 2500 == 0:
          # y2.append(model(x2))
          print("current epoch is: ", i)
          print("current loss is: ", loss.detach())

    time_end = time.time()
    print('total time is: ', time_end-time_start, 'seconds')
    loss_record_all[1].append(loss_record)
    error_record_all[1].append(error_record)
    # plot_heatmap("boundary-included",initial=False, boundary=True)


    net = Net(input_width,layer_width, layer_depth).to(device)
    initialize_weights(net)
    optimizer = optim.Adam(net.parameters())
    # %%
    loss_record = np.zeros(epoch)
    error_record = np.zeros(epoch)
    time_start = time.time()
    for i in range(epoch):
        # scale_y[i] = float(model(scale_x))
        optimizer.zero_grad()
        loss = loss_function(x1, x2, x3, initial = True, boundary=False)
        loss_record[i] = float(loss)
        loss.backward()
        optimizer.step() 
        error_record[i] = cal_error(test_data, initial=True, boundary=False)
        torch.cuda.empty_cache() # clear memory
        print("Now epoch number is:{}".format(i),end='\r')
        if i % 2500 == 0:
          # y2.append(model(x2))
          print("current epoch is: ", i)
          print("current loss is: ", loss.detach())
    time_end = time.time()
    print('total time is: ', time_end-time_start, 'seconds')
    # plot_heatmap("initial-included",initial=True, boundary=False)
    loss_record_all[2].append(loss_record)
    error_record_all[2].append(error_record)

  with open('1D_burger_loss_record_all_nu={}width=300.pkl'.format(nu), 'wb') as f:
      pickle.dump([loss_record_all,error_record_all], f)

  avg_loss_record = []
  avg_error_record = []
  var_error_record = []
  var_loss_record = []
  for i in range(len(loss_record_all)):
      avg_loss_record.append(np.mean(loss_record_all[i], axis=0))
      avg_error_record.append(np.mean(error_record_all[i], axis=0))
      var_error_record.append(np.var(error_record_all[i], axis=0))
      var_loss_record.append(np.var(loss_record_all[i], axis=0))

  plot_loss_epoch(avg_loss_record, avg_error_record, epoch,var_loss_record, var_error_record, nu)





300
current nu_list: 0.01
current run: 0
current epoch is:  00
current loss is:  tensor(1.9909)
current epoch is:  25000
current loss is:  tensor(0.0002)
current epoch is:  50000
current loss is:  tensor(0.0004)


In [ ]:
import pickle
with open('1D_burger_loss_record_all_nu=1.pkl', 'rb') as f:
    a = pickle.load(f)
loss_record_all = a[0]
error_record_all = a[1]
avg_loss_record = []
avg_error_record = []

var_error_record = []
var_loss_record = []
for i in range(len(loss_record_all)):
    avg_loss_record.append(np.mean(loss_record_all[i], axis=0))
    avg_error_record.append(np.mean(error_record_all[i], axis=0))
    var_error_record.append(np.var(error_record_all[i], axis=0))
    var_loss_record.append(np.var(loss_record_all[i], axis=0))

plot_loss_epoch(avg_loss_record, avg_error_record, epoch,var_loss_record, var_error_record,1)